## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK. В идеальном мире, вы сделаете вот так:

In [9]:
%pip install --upgrade yandex-cloud-ml-sdk

I0000 00:00:1742470000.768423    3331 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: yandex-cloud-ml-sdk
    Found existing installation: yandex-cloud-ml-sdk 0.4.1
    Uninstalling yandex-cloud-ml-sdk-0.4.1:
      Successfully uninstalled yandex-cloud-ml-sdk-0.4.1

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


В нашем случае мы установим самую последнюю версию SDK:

In [6]:
%pip install flit
%pip install git+https://github.com/yandex-cloud/yandex-cloud-ml-sdk@5d77a3618838cc4ca0553a777a39a542f900e7c8

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/yandex-cloud/yandex-cloud-ml-sdk (to revision 5d77a3618838cc4ca0553a777a39a542f900e7c8) to /tmp/pip-req-build-fuvbe4ro
  Running command git clone --filter=blob:none --quiet https://github.com/yandex-cloud/yandex-cloud-ml-sdk /tmp/pip-req-build-fuvbe4ro
  Running command git rev-parse -q --verify 'sha^5d77a3618838cc4ca0553a777a39a542f900e7c8'
  Running command git fetch -q https://github.com/yandex-cloud/yandex-cloud-ml-sdk 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Running command git checkout -q 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Resolved https://github.com/yandex-cloud/yandex-cloud-ml-sdk to commit 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Preparing metadata (pyproject.t

Также необходимо обновить некоторые библиотеки:

In [53]:
%pip install --upgrade pydantic

I0000 00:00:1743425416.188163    3524 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.0 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
inflect 6.0.5 requires pydantic<2,>=1.9.1, but you have pydantic 2.11.1 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] T

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [2]:
import os 
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']


sdk = YCloudML(folder_id=folder_id,auth=api_key)
model = sdk.models.completions("yandexgpt",model_version='rc')

In [8]:
model.run("Какое вино можно пить с устрицами?").text

'Выбор вина к устрицам зависит от ваших личных предпочтений и вида устриц. Вот несколько популярных вариантов:\n\n1. **Шардоне** — особенно из прохладных климатических зон, таких как Шабли во Франции или некоторые регионы Новой Зеландии. Эти вина обычно имеют яркую кислотность и ноты цитрусовых фруктов, которые хорошо сочетаются с солоноватостью и текстурой устриц.\n\n2. **Совиньон Блан** — особенно из регионов, таких как Марльборо в Новой Зеландии или Пойяк во Франции. Совиньон Блан обладает яркой кислотностью и свежими фруктовыми нотами, которые могут подчеркнуть вкус устриц.\n\n3. **Мюскаде** — белое вино из региона Луары во Франции, которое часто рекомендуют к устрицам. Оно обладает лёгким, свежим вкусом с нотами зелёного яблока и цитрусовых.\n\n4. **Альбариньо** — белое вино из региона Риас Байшас в Испании. Оно известно своей свежестью, высокой кислотностью и нотами экзотических фруктов, которые могут хорошо сочетаться с устрицами.\n\n5. **Шампанское и другие игристые вина** — их

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [102]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model,tools=None):
    if tools:
        return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active",tools=tools)
    else:
        return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active")

In [10]:
thread = create_thread()
assistant =  create_assistant(model)

assistant.update(instruction="Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.")

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Здравствуйте! Чтобы подобрать наиболее подходящий вариант, мне нужно знать некоторые детали:

1. Какой тип вина вы предпочитаете: красное, белое, розовое, игристое?
2. Какой стиль вина вам нравится: лёгкое и свежее или более насыщенное и крепкое?
3. Есть ли у вас предпочтения по цене или бюджету на покупку вина?
4. С какими блюдами вы планируете сочетать вино?
5. Есть ли у вас опыт с винами определённых стран или регионов, которые вам особенно понравились?

Чем больше информации вы предоставите, тем точнее я смогу подобрать рекомендации для вас.


In [11]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Для стейка я могу порекомендовать несколько вариантов вин:

* **Красное вино с насыщенным вкусом и танинностью.** Например, каберне совиньон, мерло или шираз. Эти сорта хорошо сочетаются с мясом благодаря своей структуре и способности подчёркивать вкус стейка.
* **Вина из сортов темпранильо или монтепульчано.** Они обладают хорошей кислотностью и умеренной танинностью, что делает их отличным дополнением к стейку.
* **Бордо или другие вина из региона Бордо.** Они известны своим сложным вкусом и способностью сочетаться с различными блюдами, включая стейки.

Если у вас есть предпочтения по цене или вы хотите узнать о винах из определённых регионов, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации.


In [12]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будем использовать текстовую базу знаний по винам и винным регионам, которая хранится в виде множества файлов в директориях `source/wines` и `source/regions`. Пройдёмся по этим файлам и посмотрим на их длину в токенах.

In [5]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename,'r',encoding='utf8') as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename,encoding='utf-8') as f:
        l = len(f.read())
    return l
    
d = [
        {
            "File" : fn,
            "Tokens" : get_token_count(fn),
            "Chars" : get_file_len(fn),
            "Category" : fn.split('/')[1]
        } 
    for fn in glob('source/*/*.md')
    if fn.count('/') == 2
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars,Category
0,source/regions/Абруццо.md,499,2022,regions
1,source/regions/Азорские острова.md,409,1809,regions
2,source/regions/Аконкагуа.md,278,1182,regions
3,source/regions/Алентежу.md,320,1216,regions
4,source/regions/Апулия.md,489,1918,regions
...,...,...,...,...
125,source/wines/Совиньон блан.md,634,2578,wines
126,source/wines/Темпранильо.md,637,2323,wines
127,source/wines/Цвайгельт.md,663,2620,wines
128,source/wines/Шардоне.md,567,2407,wines


Посмотрим на среднюю, мин и макс длину фрагментов:

In [14]:
df.groupby('Category').agg({ 'Tokens' : ('min','mean','max') })

Tokens                 
            min        mean  max
Category                        
regions     153  423.340000  683
wines       264  551.366667  664

Мы видим, что фрагменты не превышают 700 токенов, и это значит, что нам не придётся прибегать к какой-либо стратегии чанкования. Это идеальная ситуация, когда текстовая база знаний разбита вручную на небольшие фрагменты текста.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [6]:
def upload_file(filename):
    return sdk.files.upload(filename,ttl_days=5,expiration_policy="static")

df["Uploaded"] = df['File'].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов, но в нашем случае она не будет применяться.

> Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и по регионам по-очереди.

In [7]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy)

op = sdk.search_indexes.create_deferred(
    df[df["Category"]=="wines"]["Uploaded"],
    index_type = HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens = 1000,
            chunk_overlap_tokens = 100),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy()))
index = op.wait()

In [8]:
op = index.add_files_deferred(df[df["Category"]=="regions"]["Uploaded"])
xfiles = op.wait()

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [9]:
search_tool = sdk.tools.search_index(index)

assistant = create_assistant(model, tools=[search_tool])
thread = create_thread()

instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

In [10]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)

К стейку подойдут следующие вина:

1. **Из региона Франшхук (ЮАР):** красные сорта, такие как каберне совиньон, шираз и мерло, а также пинотаж.
2. **Из региона Сицилия (Италия):** черасуоло ди виттория (Cerasuolo di Vittoria).
3. **Из региона Орегон (США):** пино нуар.
4. **Сира (шираз):** особенно хорошо дополняет блюда из красного мяса, например, стейки. Вина из сорта сира, произведённые в прохладной Франции, обладают землистым, травяным характером, а шираз из Австралии более фруктовый, с нотами ягод и специй.


Посмотрим, из каких источников был получен этот ответ:

In [11]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != 'filechunk':
                continue
            print('------------------------')
            print(source.parts[0])
            
print_citations(result)

------------------------
## Сицилия 

Сицилия
Сицилия — регион на юге Италии. В его состав входит одноименный и самый большой в Средиземном море остров и несколько более мелких островов рядом с ним. На восточном побережье Сицилии возвышается Этна — самый высокий действующий вулкан в Европе. Помимо итальянского в регионе говорят на сицилийском языке. Административный центр Сицилии — Палермо. Это самый южный из винодельческих регионов Италии. Здесь около 120 тысяч га виноградников. С начала нулевых на Сицилии бурно развивается производство вулканических вин. Их производят из винограда, выросшего у подножия Этны. Пепел самого высокого действующего вулкана Европы богат микроэлементами, поэтому местные почвы очень плодородны. И, главное, в таких условиях нет шанса у филлоксеры. К наиболее значимым местным сортам относят красный неро д'авола. Также на Сицилии популярны красные гренаш, перриконе и ночера. Из белых стоит отметить катаратто — его смешивают с сортами грилло и инзолия и производя

In [12]:
thread.delete()

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `source/food_wine_table.md` в формате markdown.

In [13]:
with open('source/food_wine_table.md',encoding='utf-8') as f:
    food_wine = f.readlines()
fw = ''.join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12630, 3.3103721298495645 chars/token


Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [14]:
header=food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [15]:
chunk_size = 600 * 3 # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len(''.join(s))>chunk_size:
        id = sdk.files.upload_bytes(''.join(s).encode(),ttl_days=5,expiration_policy="static")
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 22 table chunks


Теперь добавим эти фрагменты в индекс:

In [16]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

Посмотрим, стал ли ответ системы лучше:

In [26]:
thread = create_thread()

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)
print_citations(result)

Выбор вина к стейку зависит от степени его прожарки и жирности.

* Для нежного мраморного стейка (например, филе-миньон) подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

* К жирноватому стейку (например, Рибай) с прожаркой Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.

* Если стейк прожарен до Medium или WellDone, то стоит выбрать сухие и полусухие вина из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше

In [22]:
thread.delete()
assistant.delete()

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [17]:
pl = pd.read_excel('source/wine-price.xlsx')
pl

,Номер артикула,название,CT,цена от 1 бутылки,от 3-х и более,регулярная цена/промо
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Для удобства переименуем колонки:

In [18]:
pl.columns=['Id','Name','Country','Price','WHPrice','etc']

Из имени извлечём кислотность вина:

In [52]:
acid_map = { 'СХ' : 'Сухое', 'СЛ' : 'Сладкое', 'ПСХ' : 'Полусухое', 'ПСЛ' : 'Полусладкое' }
pl['Acidity'] = pl['Name'].apply(lambda x: acid_map.get(x.split()[-1].replace('КР',''),''))
pl['Acidity'].value_counts()

Сухое          614
Полусухое       68
Полусладкое     28
Сладкое         23
                19
Name: Acidity, dtype: int64

Тоже самое сделаем с цветом:

In [53]:
pl['Color'] = pl['Name'].apply(lambda x: 'Красное' if (x.split()[-1].startswith("КР") or x.split()[-2]=="КР") else '')
pl['Color'].value_counts()

Красное    739
            13
Name: Color, dtype: int64

Чтобы научиться отвечать на вопросы по прайс-листу, у нас есть несколько возможных решений:

* Попытаться закинуть прайс-лист в контекст модели с помощью RAG, как мы это ранее делали с таблицей блюд. Не очень хороший вариант, потому что модель не будет видеть всю таблицу, и не сможет отвечать на вопросы "максимальная/минимальная цена".
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

ML SDK позволяет нам упростить function calling, и вместо JSON-схемы использовать типизированные объекты Pyton. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [82]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Search for a specific wine and/or its price, specified using one of parameters""" 
    name : str = Field(description="Wine name",default=None)
    country : str = Field(description="Wine Country",default=None)
    acidity : str = Field(description="Wine Acidity (сухое, полусухое, сладкое, полусладкое)",default=None)
    color : str = Field(description="Цвет вина (красное, белое, розовое)",default=None)
    sort_order : str = Field(description="What to look for (most expensive, cheapest, random, average)",default=None)
    what_to_return : str = Field(description="What to return (wine info or price)",default=None)

Теперь создадим инструмент (tool) и нового ассистента, у которого в списке инструментов будет одновременно и RAG-поиск, и function calling. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [107]:
price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model) # tools=[price_list_search_tool]
thread = create_thread()

instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций, либо, если пользователь хочет узнать про
конкретное вино или стоимость, вызови Function Calling. 
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

_ = assistant.update(instruction=instruction)

Pf

In [108]:
thread.write("Привет! Какое самое дешевое красное сухое вино из Австралии?")
op = assistant.run(thread)
res = op.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='Function Calling', arguments={'function_name': 'search_wines', 'parameters': {'limit': 1.0, 'sort_by': 'price', 'origin': 'Australia', 'style': 'dry', 'type': 'red'}})),), _message=None, usage=Usage(input_text_tokens=122, completion_tokens=40, total_tokens=162))

In [105]:
country_map = {
    'IT' : 'Италия', 'FR' : 'Франция', 'ES' : 'Испания',
    'RU' : 'Россия', 'PT' : 'Португалия', 'AR' : 'Армения',
    'CL' : 'Чили', 'AU' : 'Австрия', 'GE' : 'Грузия',
    'ZA' : 'ЮАР', 'US' : 'США', 'NZ' : 'Новая Зеландия',
    'DE' : 'Германия', 'AT' : 'Австрия', 'IL' : 'Израиль',
    'BG' : 'Болгария', 'GR' : 'Греция' }

revmap = { v.lower() : k for k,v in country_map.items() }

def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x['Country']==revmap[req.country.lower()]]
    if req.acidity:
        x = x[x['Acidity']==req.acidity.capitalize()]
    if req.color:
        x = x[x['Color']==req.color.capitalize()]
    if req.name:
        x = x[x['Name'].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order:
        if req.sort_order=='cheapest':
            x = x.sort_values(by='Price')
        elif req.sort_order=='most expensive':
            x = x.sort_values(by='Price',ascending=False)
        else:
            pass
    return "Вот какие вина были найдены:\n" + '\n'.join([
        f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}" for _,z in x.head(10).iterrows()
    ])

print(find_wines(SearchWinePriceList(country='Россия',sort_order='cheapest')))

Вот какие вина были найдены:
0,187ВИНО АВТОРСКОЕ №1 КР СХ (Россия) - 119.0
0,187Л ВИНО АДАГУМ КБСВ КР СХ (Россия) - 119.004
0,7ВИНО КАГОР ТАМ КР СЛ (Россия) - 179.004
0,7ВИНО ТАМАНИ МЕРЛО КР ПСЛ (Россия) - 229.0
0,7ВИНО ТАМАНИ КАБЕРНЕ КР СХ (Россия) - 229.0
0,75ВИНО САУК ДЕРЕ МРЛ КР СХ (Россия) - 239.0
0,75Л ВИНО АВТОРСКОЕ КАГОР КР СЛ (Россия) - 249.0
0,75ВИНО САУК ДЕРЕ КАБ КР СХ (Россия) - 299.004
0,75ВИНО ФАНАГОРИЯ КАГОР КР СЛ (Россия) - 319.0
0,75ВИНО ФАНАГОРИЯ НР САП КР ПСЛ (Россия) - 319.0


In [106]:
if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Processing function call fn={f.function.name}")
        x = SearchWinePriceList.model_validate(f.function.arguments)
        x = find_wines(x)
        result.append({ "name" : f.function.name, "content"  : x })
    print(result)
    run.submit_tool_results(result)
    res = run.wait()
res

 + Processing function call fn=SearchWinePriceList
[{'name': 'SearchWinePriceList', 'content': 'Вот какие вина были найдены:\n0,187ВИНО АВТОРСКОЕ №1 КР СХ (Россия) - 119.0\n0,187Л ВИНО АДАГУМ КБСВ КР СХ (Россия) - 119.004\n0,187ВИНО ДЖАСТ МЕРЛО КР СХ (Франция) - 149.004\n0,187ВИНО ДЖАСТ КБСВ КР СХ (Франция) - 149.004\n0,7ВИНО ТАМАНИ КАБЕРНЕ КР СХ (Россия) - 229.0\n0,75ВИНО САУК ДЕРЕ МРЛ КР СХ (Россия) - 239.0\n0,187ВИНО АНТАНЬО КР СХ (Испания) - 239.0\n0,187ВИНО КАМПО ВЬЕХ ТЕМП КРСХ (Испания) - 259.0\n0,75ВИНО АУРВИН РЕЗЕРВ КБСВ КР СХ (Неизвестно) - 299.004\n0,75ВИНО АУРВИН РЕЗЕРВ МЕРЛО КР СХ (Неизвестно) - 299.004'}]


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvtttgqsaf7k1ss9sun9', parts=('К стейку подойдут следующие вина:\n\n1. **Из региона Франшхук (ЮАР):** красные сорта, такие как каберне совиньон, шираз и мерло, а также пинотаж.\n2. **Из региона Сицилия (Италия):** черасуоло ди виттория (Cerasuolo di Vittoria).\n3. **Из региона Орегон (США):** пино нуар.\n4. **Сира (шираз):** особенно хорошо дополняет блюда из красного мяса, например, стейки. Вина из сорта сира, произведённые в прохладной Франции, обладают землистым, травяным характером, а шираз из Австралии более фруктовый, с нотами ягод и специй.',), thread_id='fvtghc0d72l5b05b6rhp', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 3, 31, 12, 53, 47, 579352), labels=None, author=Author(id='fvtuek2aqlkau4g0a8s7', role='ASSISTANT'), citations=(Citation(sources=(FileChunk(parts=("## Сицилия \n\nСицилия\nСицилия — регион на юге Италии. В его состав входит одноименный и самый б

In [91]:
for msg in thread:
    print(msg.text)
    print('-----------------------')

Привет! Какое самое дешевое красное сухое вино из Австралии?
-----------------------


In [81]:
thread.delete()
thread = create_thread()
#assistant.delete()

## Многоагентное тестирование

In [23]:
class ABot:
    def __init__(self,assistant=None,instruction=None):
        if assistant:
            self.assistant = assistant
        else:
            self.assistant = create_assistant(model)
        
        if instruction:
            self.assistant.update(instruction=instruction)

        self.thread = sdk.threads.create(name="Test", ttl_days=1, expiration_policy="static")
                    
    def __call__(self, message):
        self.thread.write(message)
        run = self.assistant.run(self.thread)
        res = run.wait()
        return res.text

    def history(self):
        return [m for m in self.thread]
    
    def restart(self):
        self.thread.delete()
        self.thread = sdk.threads.create(name="Test", ttl_days=1, expiration_policy="static")

    def done(self,delete_assistant=False):
        self.thread.delete()
        if delete_assistant:
            self.assistant.delete()


In [24]:
instruction_user = """
Ты - посетитель ресторана, и тебе нужно выбрать вино для ужина. Ты ничего не понимаешь 
в винах, и хочешь побольше распросить сомелье о разных вариантах. Ты хотел бы съесть салат
капрезе и стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. Говори
простым языком, короткими фразами. Когда ты удовлетворём выбором, скажи СПАСИБО.
"""

wine_expert = ABot(assistant = assistant)
user = ABot(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"

for i in range(10):
    print(f"Посетитель: {msg}")
    msg = wine_expert(msg)
    print(f"Сомелье: {msg}")
    msg = user(msg)
    if "СПАСИБО" in msg:
        break


Посетитель: Добрый день! Поможете мне выбрать вино?
Сомелье: Здравствуйте! С удовольствием помогу вам с выбором вина. Для начала мне нужно узнать, к какому блюду вы хотите подобрать вино. Также важно учесть ваши вкусовые предпочтения. Пожалуйста, расскажите немного подробнее о ваших планах, и я смогу дать более точные рекомендации.
Посетитель: Здравствуйте! Я хочу заказать салат капрезе и стейк. Но я не разбираюсь в винах и готов изменить меню, если вы посоветуете хорошее сочетание. Можете рассказать, какие вина подходят к разным блюдам?
Сомелье: Здравствуйте! 

Для салата капрезе я могу порекомендовать белое вино Пти Арвин Флер из региона Валле-д’Аоста в Италии. Его нежный цветочный вкус и цитрусовые нюансы прекрасно дополнят моцареллу и помидоры в вашем салате.

К стейку подойдёт красное вино, например, Фумин из того же региона Валле-д’Аоста. Это вино с ярким вкусом и нотами сладких специй, фруктов и кожи, которое отлично дополнит мясо.

Если вы хотите попробовать что-то более класси

In [66]:
assistant.delete()
index.delete()
for f in df['Uploaded']:
    f.delete()
for f in uploaded_foodwine:
    f.delete()